# Gerrymandering and the Redistricting Problem

## Assumptions

We assume a 2-party system with the parties represented as "D" and "R". We start by only using one period's worth of historical data. We also assume that all available voters voted (no one abstained) in our historical data.

## Data Inputs

The expected number of votes for each party is a key data input for our constraints. We represent this with a matrix, $\mathbf{V}$. Since we have two parties, the matrix has the following shape.

$$ \mathbf{V} \in \mathbb{R}^{|blocks| \times 2} $$

$blocks$ is a set where $|blocks|$ is the number of elements in the set. By convention, the first column of $\mathbf{V}$ will be D votes and the second column R votes. $\mathbf{V}$ could represent a single past election's results, or it could be an expected upcoming result based on an exogenous model.

## Variables

We represent the variables with a matrix. Each row of the matrix represents an indivisible block (precint, county, or census area depending on the conventions of the problem). The columns represent assignment to a district. The matrix is made up of zeroes or ones, and each row must have exactly one entry equal to one, meaning that each row must be in one and only one district. $districts$ is a set where $|districts|$ is the number of elements in each set. 

$$ 
\mathbf{D} \in \{0,1\}^{|blocks| \times |districts|}. 
$$

For ease of interpretation, we're going to define another variable that represents the "efficiency gap" in votes. To do this, we need to know the number of expected votes for each party in each block and the assignment of blocks to districts. $\mathbf{D}^T \mathbf{V} \in \mathbb{R}^{|districts| \times 2} $ gives us a matrix of the number of D and R votes in each district. The efficiency gap can be calculated from the seat margin and vote margin. [Brennan Center](https://www.brennancenter.org/sites/default/files/legal-work/How_the_Efficiency_Gap_Standard_Works.pdf)

$$ \text{Efficiency Gap} = (\text{Seat Margin} - 50\%) - 2 ( \text{Vote Margin} - 50\% ) $$

The vote margin is easily calculated by multiplying $\mathbf{D^TV}$ by a vector of ones, $\mathbf{j}$, and subtracting the elements. The seat margin is trickier, since we need to determine which of the two parties is the winner for any given potential solution.  We do this by introducing another vector of variables $\mathbf{w}\in \{0,1\}^{|districts|}$, indicating if D won the given seat (or district). 

$$ \mathbf{w}\in \{0,1\}^{|districts|} $$

When we include $w$ in our objective with an appropriately large penalty, the optimization problem will set $w_i = 1$ if D won the seat and $0$ otherwise. Our constraints will ensure this. So the margin variables are now: 

$$ \text{Seat Margin} = \frac{ (\mathbf{w}\cdot \mathbf{j} - (1-\mathbf{w})\cdot \mathbf{j} )}{|districts| } $$

$$ \text{Vote Margin} = \frac{\mathbf{j^TD^TV} \left[ \begin{array}{c} 1 \\ -1 \end{array} \right]}{\mathbf{j^TVj}} $$

In a given district's race, the number of wasted votes can be calculated szz


## Constraints

To force the $\mathbf{w}$ variable, set a constraint (vector valued):
$$ \mathbf{D^TV} \left[ \begin{array}{c} 1 \\ -1 \end{array} \right] \geq \mathbf{w} $$. 




## Example Problem

In [2]:
# These are the votes:  blocks x number of parties
V = [75 25; 60 40; 43 57; 48 52; 49 51]


5×2 Array{Int64,2}:
 75  25
 60  40
 43  57
 48  52
 49  51

In [46]:
# This is the contiguity matrix. C_{m,n} = 1 if block m shares a border with block n
C = [ 
    1 1 1 0 0;
    1 1 1 1 0;
    1 1 1 1 1;
    0 1 1 1 1;
    0 0 1 1 1;]

## So the example looks like this:
## |----------|
## |A    | B  |
## |-----|    |
## |     |----|
## | C   |    |
## |     | D  |
## |     |----|
## |     | E  |
## |-----|----|


5×5 Array{Int64,2}:
 1  1  1  0  0
 1  1  1  1  0
 1  1  1  1  1
 0  1  1  1  1
 0  0  1  1  1

In [47]:
using JuMP 
using AmplNLWriter
using CoinOptServices
using ECOS
using GLPKMathProgInterface
using NLopt

In [52]:
blocks = size(V,1)
districts = 2
t = ones(districts,1)
total_vote = V * ones(2,1)
# votes_to_win = ceil.(total_vote / 2 + 0.1)

#m = Model(solver = AmplNLSolver(CoinOptServices.bonmin, [
#    "bonmin.pump_for_minlp=yes"]) )
#m = Model(solver = ECOSSolver() )
m = Model(solver = GLPKSolverMIP())
# m = Model(solver=NLoptSolver(algorithm=:LD_SLSQP))

## Variables 

@variable(m, 0 <= D[i=1:blocks,j=1:districts] <= 1 , Bin)
@variable(m, 0 <= w[i=1:districts] <= 1, Bin)
@variable(m, min_vote[i=1:districts])
@variable(m, votes_to_win[i=1:districts])
@variable(m, eff_gap)
@variable(m, abs_eff_gap)
@variable(m, wasted_u[i=1:districts, j=1:2])
@variable(m, wasted_o[i=1:districts, j=1:2])

@variable(m, common_size)

## Constraints

# @constraint(m, D .== eye(districts))

@constraint(m, D * ones(districts,1) .== 1)  # each block can be in only one district

# These constraints set w to 1 if D wins in a district
M = districts * sum(total_vote) * 1000
@constraint(m, min_vote .>= (D' * V)[:,1] - M * (w) )
@constraint(m, min_vote .>= (D' * V)[:,2] - M * (1-w) )

# These constraints set wasted_u to the number of wasted votes for the losing parties
@constraint(m, wasted_u .>= 0)
@constraint(m, wasted_u[:,1] .>= (D' * V)[:,1] - M * w)
@constraint(m, wasted_u[:,2] .>= (D' * V)[:,2] - M * (1-w))

# These constaints set wasted_o to the number of wasted votes for the winning parties
@constraint(m, votes_to_win .== (D' * V) * [1;1] / 2)
@constraint(m, wasted_o .>= 0)
@constraint(m, wasted_o .>= (D' * V) - votes_to_win * [1 1])

# These constraints calculate the efficiency gap
@constraint(m, eff_gap .== ones(2,districts) * (wasted_u + wasted_o) * [1;-1])
@constraint(m, abs_eff_gap >= eff_gap)
@constraint(m, abs_eff_gap >= - eff_gap)

# These constraints enforce roughly equal sizes. We may want to move these to the objective
common_size_wiggle = 0.2
@constraint(m, (D' * V) * [1;1] .>= common_size * (1-common_size_wiggle))
@constraint(m, (D' * V) * [1;1] .<= common_size * (1+common_size_wiggle))


# These constraints enforce contiguity
@constraint(m, C * D .>= 2 * D)

## Objective

@objective(m, Min, abs_eff_gap + sum(min_vote) + sum(wasted_u) + sum(wasted_o) ) # abs_eff_gap + sum(w) )

m

Minimization problem with:
 * 45 linear constraints
 * 27 variables: 12 binary
Solver is GLPKInterfaceMIP

In [53]:
@time begin
    status = solve(m)
end

  0.005221 seconds (251 allocations: 40.219 KiB)


:Optimal

In [54]:
println("objective: ", getobjectivevalue(m))


_D = getvalue(D)
_D' * V


objective: 459.0


2×2 Array{Float64,2}:
 183.0  117.0
  92.0  108.0

In [56]:
getvalue(wasted_u)

2×2 Array{Float64,2}:
 -0.0  117.0
 92.0   -0.0

In [57]:
# size of each district
print("common size: ", getvalue(common_size))
(_D' * V) * [1;1]


common size: 250.0

2-element Array{Float64,1}:
 300.0
 200.0

In [58]:
getvalue(wasted_o)

2×2 Array{Float64,2}:
 33.0  -0.0
 -0.0   8.0

In [59]:
getvalue(eff_gap)

0.0

In [60]:
_D

5×2 Array{Float64,2}:
 1.0  0.0
 1.0  0.0
 0.0  1.0
 1.0  0.0
 0.0  1.0

In [101]:
V = [75 25; 60 40; 43 57; 48 52; 49 51]

D = eye(5)
(D' * V) 
ceil.(V * ones(2,1) / 2 + 0.1)
V
D' * V

5×2 Array{Float64,2}:
 75.0  25.0
 60.0  40.0
 43.0  57.0
 48.0  52.0
 49.0  51.0

In [20]:
V[:,1] .* [1;2;3;4;5]

5-element Array{Int64,1}:
  75
 120
 129
 192
 245